<a href="https://colab.research.google.com/github/skywalker0803r/deep-learning-ian-goodfellow/blob/master/%E9%9B%B2%E7%AB%AF%E8%99%9B%E6%93%AC%E8%B2%A8%E5%B9%A3%E9%80%B2%E5%87%BA%E5%A0%B4%E6%8F%90%E9%86%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib
import finlab_crypto
import pandas as pd

def send_message(title,text,email='skywalker0803r@gmail.com',password="fbkafkfgjrddzofj"):
  content = MIMEMultipart()
  content["subject"] = title
  content["from"] = email
  content["to"] = email
  content.attach(MIMEText(text))
  with smtplib.SMTP(host="smtp.gmail.com", port="587") as smtp:
    smtp.ehlo()
    smtp.starttls()
    smtp.login(email,password)
    smtp.send_message(content)

def main(n1=10,n2=20,symbol='ETHUSDT'):
  ohlcv = finlab_crypto.crawler.get_all_binance(symbol,'4h')
  table = pd.DataFrame()
  table['close'] = ohlcv.close
  table['n1'] = ohlcv.close.rolling(n1).mean()
  table['n2'] = ohlcv.close.rolling(n2).mean()
  table['buy'] = ((table['n1']>table['n2'])&(table['n1'].shift()<table['n2'].shift())).astype(int)
  table['sell'] = ((table['n1']<table['n2'])&(table['n1'].shift()>table['n2'].shift())).astype(int)
  
  if table['buy'][-1] == 1:
    message = 'BUY'
  
  elif table['sell'][-1] == 1:
    message = 'SELL'
  
  else:
    message = 'HOLD'
  
  send_message(title=message,text='Action:{},Symbol:{},Price:{}'.format(message,symbol,table['close'][-1]))